## NCAA Early Applicant to NBA Picker
Today we are trying to see if we can determine whether given a USA NCAA player who's applied to the NBA, whether that person is likely to get drafted. We shall do this through a variety of techniques. Firstly we must generate our own unique dataset utilizing BigQuery's ncaa_mbb dataset. Our first issue with the ncaa_mbb dataset is we have no idea whether any of the applicants have applied to the nba. To solve this issue, we shall partition the data by adding another categorical feature to each player indicating whether this player applied to the nba. Specifically we will label whether a player is an early nba applicant, simply because early applicants seemed to be the only reliable form of data we could find regarding nba applicants. 

In [2]:
# Simple python libraries
import numpy as np
import matplotlib.pyplot as plt
import csv
from data_loader.DatasetsGenerator import getall
from data_loader.Draft import Draft
from data_loader.Applicants import Applicants
# from data_loader import Draft, DatasetsGenerator
%matplotlib inline

# Generate all custom made datasets
getall()

# Create an nba drafted validation class
draft = Draft()

# Create a nba applicant validation class
applicants = Applicants()

In [34]:
# Directories with early_applicants unparsed labels
